# Consolidación de bases 

  El código descrito a continuación tiene dos funcionalidades:

1. <p style='text-align: justify;'>La primera es descargar cada uno de las resoluciones publicadas en la página del Miniterio de Salud, con base al Scrapping realizado en el script Scrapping_Resoluciones_MinSalud.ipynb.</p>
2.  <p style='text-align: justify;'> La segunda parte, transcribe los pdf a texto y valida si los textos tienen menos de 200 palabras para trasnformar dichos pdf en archivos Word y posteriormente reescribir el archivo PDF, esto debido a que hay textos que se publican y son realizados a través de escaneos; por tanto, una manera de solucionar eso es la descrita anteriormente. </p>

La finalidad de este script es obtener cada uno de los textos de las resoluciones y generar un dataframe para exportar como excel para el analisis de texto.

En primera medida, se importan las librerias necesarias para los procesos descritos en las funcionalidades.</p> 

In [ ]:
import os
import pandas as pd
import string
import nltk
from nltk import word_tokenize
from nltk.corpus import stopwords
from tika import parser
import PyPDF3
import matplotlib.pyplot as plt
%matplotlib inline
import glob
import win32com.client
import sys
import comtypes.client
plt.rcParams["figure.figsize"] = [18.0, 8.0]

In [ ]:
links=pd.read_excel("links_resoluciones.xlsx")
links=links.drop(columns=("Unnamed: 0"))
links['años']=links['años1'].fillna(0).astype(int)
links=links[["links", "nombre", "años"]]

In [ ]:
links.head()

In [ ]:
links.dtypes

El siguiente código lo que hace es realizar el request de cada ruta de resolución y guarda el PDF de cada una en la ruta donde se encuentra este script. 

In [ ]:
import urllib.request
for i in range(0,len(links)):
    print(links['nombre'][i])
    try:
        urllib.request.urlretrieve(links['links'][i], links['nombre'][i])
    except:
        None

<p style='text-align: justify;'>  El siguiente código genera una función la cual será usada para extraer texto de cada uno de los textos a través de dos métodos, uno es usando la librería "tika", de alli, "parser" trascribe los textos de pdf a una list; sin embargo, para los archivos que dicha librería no pueda trascribir se usa la librería "PyPDF3" el cual transcribe una por una de las hojas de los PDFs.</p>

In [ ]:
def texto(archivopdf):
    try:
        rawText = parser.from_file(archivopdf)
        rawList1 = [rawText['content'].replace("\n", "")]
    except:
        try:
            pdfFileObject = open(archivopdf, 'rb')
            pdfReader = PyPDF3.PdfFileReader(pdfFileObject)
            count = pdfReader.numPages
            rawList1=[]
            
            for i in range(count):
                page = pdfReader.getPage(i)
                texto0=page.extractText()
                rawList1.append(texto0.replace("\n", ""))
        except:
            rawList1=[]
    return rawList1

El siguiente código realiza la trascipción de cada resolución usando la función anterior y guardandolo en una lista. Luego guarda esa lista en una columna de la base de links de resolución publicados por el Ministerio de Salud.

In [ ]:
textos_transcritos=[]
for i in range(len(links)):
    print(i)
    palabras=texto(links['nombre'][i])
    textos_transcritos.append(palabras)

In [ ]:
links['textos']=textos_transcritos

El siguiente código realiza el conteo de cada palabra por cada resolución y lo guarda en una columnda del dataframe.

In [ ]:
links['numero_palabras']=''
for i in range(len(textos_transcritos)):  
    try:
        links['numero_palabras'][i]=len(textos_transcritos[i][0])
    except:
        links['numero_palabras'][i]=0

Para los archivos que transcribio y no supero las 500 palabras, se realiza el proceso de convertir dichos archivos a Word y posteriormente gurda como PDF, reemplazando el PDF inicial.

In [ ]:
link1=links[links['numero_palabras']<=500].reset_index(drop=True)

In [ ]:
link1.head()

Las siguientes funciones realizan las siguientes tareas:
1. A través de la función "pdf_a_word", con el uso la librería "win32com" transforma los pdf en archivos word y los guarda en la misma ruta de este script.
2. A través de la función "eliminacion" elimina el pdf que se guardó como word con la función del punto 1.
3. A través de función "word_pdf", guarda el word creado en el punto 1 como PDF con el mismo nombre del archivo eliminado en el punto 2.

In [ ]:
def pdf_a_word(archivo):
    word = win32com.client.Dispatch("Word.Application")
    word.visible = 0
    pdfs_path = archivo
    
    for i, doc in enumerate(glob.iglob(pdfs_path)):
        filename = doc.split('\\')[-1]
        in_file = os.path.abspath(doc)
        
        print("pdf transformado:\n",in_file)
        wb = word.Documents.Open(in_file)
        out_file = os.path.abspath(filename[0:-4]+ ".docx".format(i))
        
        print("word transformado a pdf:\n",out_file)
        wb.SaveAs2(out_file, FileFormat=16) 
        wb.Close(True)# cerrar la aplicación word
    word.Quit()

In [ ]:
def eliminacion(ruta):
    if os.path.isfile(ruta) or os.path.islink(ruta):
        os.remove(ruta)  # 
    elif os.path.isdir(ruta):
        shutil.rmtree(ruta)  
    else:
        raise ValueError("Archivo {} no esta en la direccion.".format(ruta))

In [ ]:
def word_pdf(archivo):
    wdFormatPDF = 17

    in_file = os.path.abspath(archivo[:-4]+".docx")
    out_file = os.path.abspath(archivo[:-4]+".pdf")

    word = comtypes.client.CreateObject('Word.Application')
    doc = word.Documents.Open(in_file)
    doc.SaveAs(out_file, FileFormat=wdFormatPDF)
    doc.Close()
    word.Quit()

El siguiente código hace la iteración de las funciones descritas anteriormente para cada uno de los registros que no cumplen con una longitud de palabras mayores a 500. Esto dado que algunos PDFs fueoron creador a tarvés de un proceso de escaneo. 

In [ ]:
for i in range(0,len(link1)):
    try:
        pdf_a_word(link1['nombre'][i])
        eliminacion(link1['nombre'][i])
        word_pdf(link1['nombre'][i])
    except:
        None

El siguiente código tiene como objetivo iterar la función de transcribir de nuevo los textos para volver a validar si efectivamente hubo un aumento en el número de palabras de cada resolución. Luego de esto, guarda las trascripciones en otra columna del dataframe base que se tiene para el análisis. 

In [ ]:
textos_transcritos1=[]
for i in range(len(links)):
    print(i)
    palabras=texto(links['nombre'][i])
    textos_transcritos1.append(palabras)
links['textos1']=textos_transcritos1

El siguiente código realiza el conte de palabras en cada uno de las resoluciones publicados por el MinSalud, con base al arreglo generado anteriormente.

In [ ]:
links['numero_palabras1']=''
for i in range(len(textos_transcritos)):  
    try:
        links['numero_palabras1'][i]=len(textos_transcritos1[i][0])
    except:
        links['numero_palabras1'][i]=0

In [ ]:
links[links['numero_palabras1']<=500]

In [ ]:
links.to_excel("Textos_resoluciones.xlsx")